In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, BaseOutputParser
from textwrap import dedent

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import httpx
import json
import requests
import pandas as pd
import numpy as np
import re

from langchain_pymupdf4llm import PyMuPDF4LLMLoader, PyMuPDF4LLMParser
import pymupdf4llm
import requests
import uuid
from openai import OpenAI

import time
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))


use = 'nhf'

if use == 'hf':
    os.environ["BASE_URL"] = "https://router.huggingface.co/v1"
    os.environ["MODEL_NAME"] ='Qwen/Qwen3-32B'
    os.environ["OPENAI_API_KEY"] = os.getenv("HUGGINGFACE_HUB_TOKEN")
else:
    os.environ["BASE_URL"] = "https://gigachat.devices.sberbank.ru/api/v1"
    os.environ["MODEL_NAME"] ='GigaChat'
    
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())
    payload={
      'scope': 'GIGACHAT_API_PERS'
    }
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Accept': 'application/json',
      'RqUID': rq_uid,
      'Authorization': f'Basic {os.getenv("GIGACHAT_AUTH")}'
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    os.environ["OPENAI_API_KEY"] = response.json()['access_token']
    

os.environ['TEMPERATURE'] = '0'




from trialmind.pubmed import pmid2papers, PubmedAPIWrapper, pmid2biocxml, parse_bioc_xml
from trialmind.api import StudyCharacteristicsExtraction, ScreeningCriteriaGeneration,\
                            LiteratureScreening, ScreeningCriteriaCTGeneration,\
                            CTScreening
from trialmind.retrievers import split_text_into_chunks
import extract

%load_ext autoreload
%autoreload 2

C:\Users\user\anaconda3\envs\llms\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


C:\Users\user\anaconda3\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [2]:
os.getenv("OPENAI_API_KEY")

'eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.TGIErQuLrnDjf9xevfoXIKoh2uFmtRowz5E33MhAkTkdHqm-hLdVsbG092siPPgnhXJUMUScgCMD32L4EUq4z7fPf730K9a3j-mcMrSCHwus2VkcX8v2SPRzFvCFDOqVthcWPL2LAc_FlVc2n9trQ8ZYzjKYntHv58oyMu6qH-dTG8sV-ERMB8FPBBLQhGgYw_fFxHELfyPR8ym2XF4-dtQDMqzU9XZLpzn18_TybnhbM-KGCRLEntiyxW1uRc_qvsg_9Z6JtHHMNzb5RVEKhwwWMvXDFgVW61iVUCOQ5_Gh1KqdvQGmSQ5JpxeiTEFgbn6xIlU2nwiZ128S8Y5bpg.X5Iv5OQYEuXiyogcJbRzNQ.0qTx9nEOYtiQDmFlAu4GznUsWsBg36eNfcSGi9iNkhDpu6RIsLPyMzRE0nMp_NHa5tfQ7CLsvpJiVkKqjdGOobbHs313nRoKopuwrvrpJuafcvY7TKBAb0hGwBvQ-CsAt179_-KOCCe7IEwlFzQ8fO7wNkcImC8kyO_5I8pevvOBWvXVl1_fWgY9LmqnjoRwjJk_vmdSCKSoSRESW5tT_F-9dL7XNhNoKUplqD8PMMsOcaDgxhBiBA-QvFyMkiWzVIjHYoXyE88cOHc_NyqSmWdyMxeKjQcxwmEnusT9k0R3gffz_P3dgXERYO0gnY6H3DxLfyYZhYaDtNayLoVJvv-XP1xsOXd9JqGVwRLYI8vCMcrlLMsHHktXFiNP15nm_ict06MVlKqNMwZIiAL8TQuimts0ZK-nvTmQZkAYT9M7uLQJsmh9khbOzpsaB7evODc19f0mfpKxDQP9lVhAHDCM2ggZMd0KOTJUfEvhihx7HBtiEzZOVEL1QwG9scV0rnbthCNWPjLRffQqCvojs9Gp02ufy0w2RgQfFuq1lR0ja

In [4]:
client = OpenAI(
        base_url=os.getenv("BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        http_client=httpx.Client(verify=False)
    )
response = client.chat.completions.create(
        model=os.getenv("MODEL_NAME"),
        messages=[{'role':'user','content':'hey\no_think'}],  # Ensure messages is a list
        temperature=0,
    )
response.choices[0].message.content

'Привет! Как настроение?'

In [27]:
Results.model_json_schema()

{'$defs': {'FieldResult': {'properties': {'name': {'description': 'Field name that accurately represents the content of the field based on its description.',
     'maxLength': 25,
     'title': 'Name',
     'type': 'string'},
    'value': {'description': 'Extracted information from the text based on the field description.',
     'maxLength': 50,
     'title': 'Value',
     'type': 'string'},
    'source_id': {'description': 'Cited document IDs.',
     'items': {'type': 'integer'},
     'maxItems': 3,
     'minItems': 1,
     'title': 'Source Id',
     'type': 'array'}},
   'required': ['name', 'value', 'source_id'],
   'title': 'FieldResult',
   'type': 'object'}},
 'properties': {'fieldresult': {'$ref': '#/$defs/FieldResult'}},
 'required': ['fieldresult'],
 'title': 'Results',
 'type': 'object'}

In [30]:
Results.model_json_schema()
completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": f"Generate some examples with two fieldresult with the following JSON schema: {Results.model_json_schema()}  /no_think"
        }
    ],
    response_format={
        "type": "json_object",
        "json_schema": {
            "name": "car-description",
            "schema": Results.model_json_schema()
        },
    },
    temperature=0,
)
print(completion.choices[0].message.content)

### Example JSON data using the provided schema:

```json
{
  "fieldresult": [
    {
      "name": "Author",
      "value": "Иванов Иван Иванович",
      "source_id": [12345]
    },
    {
      "name": "Publication Date",
      "value": "1 января 2023 года",
      "source_id": [67890]
    }
  ]
}
```

#### Explanation:
- **`name`:** The descriptive label for the extracted field (e.g., “Author” or “Publication Date”).
- **`value`:** The actual value extracted from the source text (e.g., a person’s name or date).
- **`source_id`:** An array of integers representing the cited documents where this field was found.

### Additional Examples

1. **Example 1:**
   ```json
   {
     "fieldresult": [
       {
         "name": "ISBN",
         "value": "978-123456789X",
         "source_id": [101, 102]
       },
       {
         "name": "Publisher",
         "value": "ООО \"Издательство Книги и Статьи\"",
         "source_id": [103]
       }
     ]
   }
   ```

2. **Example 2:**
   ```json
   {


### client.responses дает 403 ошибку, т.е. structured output не работает

In [43]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(http2=True, verify=False)
)

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

response = client.responses.parse(
    model=os.getenv("MODEL_NAME"),
    input=[
        {
            "role": "system",
            "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure.",
        },
        {"role": "user", "content": "..."},
    ],
    text_format=ResearchPaperExtraction,
)

research_paper = response.output_parsed

PermissionDeniedError: <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>SynGX</center>
</body>
</html>

In [46]:
response = client.responses.create(
    model=os.getenv("MODEL_NAME"),
    input=[{"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
    {"role": "user", "content": "how can I solve 8x + 7 = -23"}],
    text={
    "format": {
    "type": "json_schema",
    "name": "math_response",
    "schema": {
    "type": "object",
    "properties": {
    "steps": {
    "type": "array",
    "items": {
    "type": "object",
    "properties": {
    "explanation": {"type": "string"},
    "output": {"type": "string"}
    },
    "required": ["explanation", "output"],
    "additionalProperties": False
    }
    },
    "final_answer": {"type": "string"}
    },
    "required": ["steps", "final_answer"],
    "additionalProperties": False
    },
    "strict": False
    }
    }
    )

print(response.output_text)

PermissionDeniedError: <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>SynGX</center>
</body>
</html>

In [57]:
response = client.chat.completions.create(
        model=os.getenv("MODEL_NAME"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "Who won the world series in 2020? Please respond in the format {winner: ...}"}
        ],
        text={"format": {"type": "json_object"}}
    )
print(response.choices[0].message.content)

TypeError: Completions.create() got an unexpected keyword argument 'text'

### response_format: {type: "json_schema",...} -- Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}
Structured Outputs with response_format: {type: "json_schema", ...} is only supported with the gpt-4o-mini, gpt-4o-mini-2024-07-18, and gpt-4o-2024-08-06 model snapshots and later.

In [90]:
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Ford Explorer (SUV) car from the 90's"
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "car-description",
            "schema": CarDescription.model_json_schema()
        },
    },
)
print(completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}

### back

In [43]:
from enum import Enum
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's  /no_think"
        }
    ],
    response_format={
        "type": "json_object",
        "json_schema": {
            "name": "car-description",
            "schema": CarDescription.model_json_schema()
        },
    },
    temperature=0,
)
print(completion.choices[0].message.content)

### JSON data for Mazda cars from the 1990s:

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 Miata",
    "car_type": "Roadster"
  },
  {
    "brand": "Mazda",
    "model": "Protege",
    "car_type": "Compact Sedan"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Full-size Luxury Sedan"
  },
  {
    "brand": "Mazda",
    "model": "B2300",
    "car_type": "Pickup Truck"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  },
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "Mid-size Sedan"
  }
]
```

This JSON contains information about popular Mazda models from the 1990s, including their respective brands, models, and types (sedans, roadsters, minivans, etc.).


In [ ]:
response.msgs[0].parsed

In [36]:
'''
Gigachat --  Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}
works for qwen! 
'''
from enum import Enum
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.parse(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's  /no_think"
        }
    ],
    response_format=CarDescription,
    temperature=0,
)
print(completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}

In [37]:
json_schema

{'$defs': {'CarType': {'enum': ['sedan', 'SUV', 'Truck', 'Coupe'],
   'title': 'CarType',
   'type': 'string'}},
 'properties': {'brand': {'title': 'Brand', 'type': 'string'},
  'model': {'title': 'Model', 'type': 'string'},
  'car_type': {'$ref': '#/$defs/CarType'}},
 'required': ['brand', 'model', 'car_type'],
 'title': 'CarDescription',
 'type': 'object'}

In [40]:
from enum import Enum
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"+\
        f" and follow the JSON format defined by the following JSON schema: {json_schema} /no_think"
        }
    ],
    #tools=[json_schema],
    temperature=0,
)
print(completion.choices[0].message.content)

### JSON data for Mazda cars from the 1990s

```json
[
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "sedan"
  },
  {
    "brand": "Mazda",
    "model": "MX-5",
    "car_type": "Coupe"
  },
  {
    "brand": "Mazda",
    "model": "Bongo",
    "car_type": "Van"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  },
  {
    "brand": "Mazda",
    "model": "Navajo",
    "car_type": "Pickup Truck"
  },
  {
    "brand": "Mazda",
    "model": "Protege",
    "car_type": "Hatchback"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Sedan"
  },
  {
    "brand": "Mazda",
    "model": "Tribute",
    "car_type": "SUV"
  }
]
```

### Explanation:
- The `'$defs'` section defines the allowed values for the `car_type`.
- Each entry in the array represents a different Mazda car from the 1990s.
- The required fields (`brand`, `model`, and `car_type`) are specified as per the given JSON schema.
- The `'$ref'` is used to reference the predefi

In [39]:
'''
qwen doesnt support "structured_outputs" in body
'''
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"
        }
    ],
    extra_body={"structured_outputs": {"json": json_schema}}
)
print(completion.choices[0].message.content)

### Example JSON structure for Mazda cars from the 1990s:

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 Miata",
    "car_type": "Convertible"
  },
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "Sedan/SUV"
  },
  {
    "brand": "Mazda",
    "model": "Capella",
    "car_type": "Compact Car"
  },
  {
    "brand": "Mazda",
    "model": "Bongo Freestyle",
    "car_type": "Minivan/MPV"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Mid-size Luxury Sedan"
  },
  {
    "brand": "Mazda",
    "model": "Protégé",
    "car_type": "Compact Car"
  }
]
```

#### Explanation:
- The `brand` field contains the manufacturer name (`Mazda`).
- The `model` field specifies the individual car models that were popular in the 1990s.
- The `car_type` indicates the type (sedan, convertible, minivan, etc.) to provide more context about each vehicle.

You can modify or extend this list based on specific Mazda models you want to include from the 1990s.


In [42]:
'''
qwen doesnt support "structured_outputs" in body
'''
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"
        }
    ],
    tools = [json_schema]
)
print(completion.choices[0].message.content)

### JSON format for Mazda Cars from the 1990s

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 (Miata)",
    "car_type": "Convertible sports car"
  },
  {
    "brand": "Mazda",
    "model": "Protégé",
    "car_type": "Compact sedan/coupe"
  },
  {
    "brand": "Mazda",
    "model": "B-series pickup truck",
    "car_type": "Pickup truck"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Full-size luxury sedan"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  },
  {
    "brand": "Mazda",
    "model": "Navajo",
    "car_type": "Sport utility vehicle (SUV)"
  }
]
```

This JSON contains information about popular Mazda models from the 1990s, including their brand, model name, and car type.


## 1. clinical trials

In [4]:
file_path = "docs/CC_215_L00_DL_fusion_fixed_oncobox_ru.pdf"
fin_condition, treatements_eng = extract.info_from_doc(file_path)
fin_condition, treatements_eng

('Colorectal cancer',
 ['Crizotinib',
  'Lorlatinib',
  'Necitumumab',
  'Nimotuzumab',
  'Panitumumab',
  'Cetuximab',
  'Brigatinib',
  'Ramucirumab',
  'Pomalidomide',
  'Tamoxifen',
  'Denosumab',
  'Duvelisib',
  'Inavilisib',
  'Bevacizumab',
  'Anastrozole',
  'Exemestane'])

1. загрузить док
2. взять 1-ую страницу; рег.выражениями взять диагноз
-- с ЛЛМ перевести на англ.
3. найти страницу с таблицей; или взять таблицы с PyMuPDFLoader
4. извлечь таблицу

In [5]:
all_studies = extract.get_clinicaltrials(f'''"{fin_condition}"''', 
                                      #' OR '.join(treatements_eng), 
                                       treatements_eng[0],  
                                      max_studies=10)
all_studies

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures
0,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat..."
1,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN
2,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,This phase II MATCH screening and multi-sub-tr...,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN
3,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN
4,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,The purpose of this study is to evaluate the s...,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN
5,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"This is an open-label, multicenter, global Pha...","[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN
6,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN


In [6]:
# test the apis

api = ScreeningCriteriaCTGeneration()
ecs = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=2,
    num_abstract_criteria=2,
    thinking=False,
)
ecs

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with Crizotinib as part of standard therapy or experimental protocol",
        "Comparative intervention against placebo or other treatments",
        "Clearly reported outcomes related to efficacy or safety endpoints",
        "Published in peer-reviewed journals"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'Crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a description of patient population matching the inclusion criteria?",
        "Are the outcomes specified in the abstract relevant to the study objectives?"
    ]
}
```


{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title mention 'Crizotinib'?"],
 'content_criteria': ['Is there a description of patient population matching the inclusion criteria?',
  'Are the outcomes specified in the abstract relevant to the study objectives?'],
 'eligibility_analysis': ['Patients must have colorectal cancer confirmed by histopathology or imaging studies',
  'Treatment with Crizotinib as part of standard therapy or experimental protocol',
  'Comparative intervention against placebo or other treatments',
  'Clearly reported outcomes related to efficacy or safety endpoints',
  'Published in peer-reviewed journals']}

### __check

In [7]:
from trialmind.api import parse_json_outputs

In [29]:
parse_json_outputs(['''```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with crizotinib as part of standard therapy or experimental protocol",
        "Comparative intervention against placebo or other treatments",
        "Clearly reported outcomes related to efficacy or safety endpoints",
        "Published in peer-reviewed journals"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a description of patient population including diagnosis and stage of disease?",
        "Are details provided about the intervention and comparison groups?"
]
} ```
'''])

[{}]

In [15]:
json_pattern = r"\`\`\`json([\s\S]*?)\`\`\`"
if re.search(r"\`\`\`json([\s\S]*?)", '''```json
{{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with crizotinib as part of standard therapy or experimental protocol",
        "Comparative intervention against placebo or other treatments",
        "Clearly reported outcomes related to efficacy or safety endpoints",
        "Published in peer-reviewed journals"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a description of patient population including diagnosis and stage of disease?",
        "Are details provided about the intervention and comparison groups?"
]'''):
    print(1)

In [128]:
from typing import Literal
class ScreeningResult(str, Enum):
    evaluations: Literal['YES', 'NO', 'UNCERTAIN'] = Field(description=f"Evaluation for a criteria")
    rationale: str = Field(description="A rationale for a criteria evaluation")

class ScreeningResults(BaseModel):
    one_result: ScreeningResult

json_schema = ScreeningResults.model_json_schema()
json_schema

{'$defs': {'ScreeningResult': {'enum': ["annotation=NoneType required=True description='Evaluation for a criteria'",
    "annotation=NoneType required=True description='A rationale for a criteria evaluation'"],
   'title': 'ScreeningResult',
   'type': 'string'}},
 'properties': {'one_result': {'$ref': '#/$defs/ScreeningResult'}},
 'required': ['one_result'],
 'title': 'ScreeningResults',
 'type': 'object'}

### 2. CT screen

In [7]:
all_text = all_studies.briefTitle.fillna("") + ": " + all_studies.briefSummary.fillna("")
all_text.values.tolist()[:1]

["MEK and MET Inhibition in Colorectal Cancer: This trial is designed to try two new cancer drugs together for the first time. The investigators think that they might be effective in some types of bowel cancer. The first part of the trial will see what doses of the two drugs can safely be given together. Once the investigators have identified a suitable dose combination they will look at how effective treatment is in bowel cancers where either the RAS gene is mutated, or MET is over-active. In the trial the investigators will look at samples of blood, skin and tumour to check the drugs are working in the way expected. The trial will take place in three sites in the UK and 5 sites in Europe. The trial is funded as part of the European commission's FP7 program."]

In [8]:

api = CTScreening()
ec_pred = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = ecs['title_criteria'] + ecs['content_criteria'],
    papers = all_text.values.tolist(), # make for the top-100 for demo
)
ec_pred


# CONTEXT #
You are a clinical specialist tasked with assessing clinical trials for inclusion in a meta-analysis based on specific eligibility criteria.

# OBJECTIVE #
Evaluate each criterion of a given clinical trial to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.

# IMPORTANT NOTE #
If the information within the provided clinical trial content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.

# PICO FRAMEWORK #
- P (Patient, Problem or Population): {P}
- I (Intervention): {I}
- C (Comparison): {C}
- O (Outcome): {O}

# PAPER DETAILS #
- Provided clinical trial: {paper_

[{'evaluations': ['YES', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["The title mentions 'Colorectal cancer'",
   "The title mentions 'Crizotinib'",
   'There is a description mentioning patients with colorectal cancer',
   'The outcomes mentioned in the abstract do not explicitly state relevance to Crizotinib use']},
 {'evaluations': ['YES', 'YES', 'YES', 'YES'],
  'rationale': ["Title mentions 'Colorectal cancer'",
   "Title mentions 'Crizotinib'",
   'Patient population described as having colorectal cancer with ALK mutation',
   'Abstract specifies outcomes related to efficacy and safety']},
 {'evaluations': ['NO', 'NO', 'UNCERTAIN', 'UNCERTAIN'],
  'rationale': ['Title does not mention colorectal cancer',
   'Title does not mention Crizotinib',
   'Patient population described but not specifically colorectal cancer',
   'Outcomes unspecified in provided abstract']},
 {'evaluations': ['YES', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["Title mentions 'Colorectal cancer'",
   "Title

In [9]:
evals = [i['evaluations'] for i in ec_pred]

word2int = {"YES": 1, 
            "UNCERTAIN": 0,
            "NO": -1
           }
#rev_subs = { v:k for k, v in word2int.items()} # subs.iteritems() In Python 3
new_evals = []
for one_e in evals:
    new_evals.append([word2int.get(item, 0) for item in one_e ])
new_evals = np.array(new_evals)    
print(new_evals)
print(new_evals.sum(axis=1))
all_studies['screen_eval'] = new_evals.sum(axis=1)
all_studies

[[ 1  1  1  0]
 [ 1  1  1  1]
 [-1 -1  0  0]
 [ 1  1  1  0]
 [-1 -1  0  0]
 [-1 -1  0  0]
 [ 1  1  1  1]]
[ 3  4 -2  3 -2 -2  4]


,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval
0,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",3
1,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN,4
2,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,This phase II MATCH screening and multi-sub-tr...,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN,-2
3,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN,3
4,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,The purpose of this study is to evaluate the s...,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN,-2
5,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"This is an open-label, multicenter, global Pha...","[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN,-2
6,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN,4


In [10]:
all_studies[all_studies.screen_eval>=0]

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval
0,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",3
1,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN,4
3,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN,3
6,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN,4


### 3. CT outcome results
Взяли все результаты, отдельно сохранили испытания с результатами, и з них отбираем рез-ты только с unit==participants

filter by interventional study type?

In [55]:
from trialmind.llm import batch_call_llm

In [11]:
at_once = False

chosen = all_studies[(all_studies.screen_eval>=0)&(all_studies.hasResults==True)]
chosen['res_with_part'] = chosen['outcomeMeasures'].apply(lambda x: [obj for obj in x \
                                                          if(obj.get('unitOfMeasure','').lower() in ['percentage of participants','participants'])])

to_work = chosen[chosen.res_with_part.str.len()>0]
if to_work.shape[0]:
    messages = []
    for i in to_work.res_with_part.values[:2]:
        n_outcomes = len(i)
        print(n_outcomes)
        messages.append([{'role':'system', 'content':f'You are a helpful medical assistant. Answer with a concise description of the efficiency statistics for EACH of {n_outcomes} provided outcome measures /no_think'},
                {'role':'user', 'content':f"{i}"}])
    if at_once:
        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), 
                                    int(os.getenv('TEMPERATURE')), thinking=False)
    else:
        results = []
        for i in messages:
            answer = extract.use_llm(os.getenv('MODEL_NAME'),i)
            results.append(answer)
    for i in results:
        print(i)
        print('-----')

C:\Users\user\AppData\Local\Temp\ipykernel_9776\3128703095.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen['outcomeMeasures'].apply(lambda x: [obj for obj in x \


1
**Outcome Measure Efficiency Statistics:**

1. **Complete Response (CR):** Not reported.
2. **Partial Response (PR):** Not reported.
3. **Stable Disease (SD):** 7 participants out of 30 (23.3%).
4. **Progressive Disease:** 22 participants out of 30 (73.3%).
5. **Early Death from Malignant Disease:** 1 participant out of 30 (3.3%).
-----


In [59]:
temperature=0
def results_ct(all_studies):
    chosen = all_studies[all_studies.results==True]
    chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x if(obj.get('unitOfMeasure','') == 'Participants')])
    
    to_work = chosen[chosen.res_with_part.str.len()>0].iloc[:2]
    if to_work:
        outcome_participants = [obj for item_main in chosen.outcomes.values for obj in item_main 
                            if(obj.get('unitOfMeasure','') == 'Participants')] 
        messages = []
        for i in to_work.res_with_part.values:
            messages.append([{'role':'system', 'content':'You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give: 1) a concise description of results with percentages and 2) a short description of study objectives /no_think'},
                    {'role':'user', 'content':f"{i}"}])

        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), int(os.getenv('TEMPERATURE')), thinking=False)
        return results
    else:
        return None

res = results_ct(all_studies)
for i in res:
    print(i)
    print('-----')

C:\Users\user\AppData\Local\Temp\ipykernel_17840\480369459.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x if(obj.get('unitOfMeasure','') == 'Participants')])


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give a concise description with percentages /no_think"),
    ("human", "{text}"),
    # ("ai", "Good.")
])

chain2 = prompt2 | llm_reasoning
response_res = await chain2.ainvoke({"text": outcome_participants[0]})
print(response_res.content)

NameError: name 'llm_reasoning' is not defined

### results in a given format?

In [ ]:

class Info(BaseModel):
    diagnose: str = Field(..., description="База клинических испытаний")
    treatments: str = Field(..., description="Высшие позиции рейтинга таргетных препаратов")

parser = PydanticOutputParser(pydantic_object=Info)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        dedent(
            """
            You are a helpful assistant. 
            Your goal is to extract related information from the text provided by the user.
            Return the information in the following JSON-format.
            {fmt}
            """
        )
    ),
    (
        "human", 
        dedent(
            """
            Text: {text}"
            """
        )
    ),
])
chain = (
    prompt 
    | llm_reasoning 
    | parser
)


response = await chain.ainvoke({"text": text, "fmt": parser.get_format_instructions()})
print(response)

## 4. papers

In [5]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]
pubmed_api_key= '6892c4129cef143ff92d11533848d2e0d908'#os.getenv("PUBMED_API_KEY", '')

In [6]:
search_api = PubmedAPIWrapper()
# page_size is the max number of records to return!!!! not pages!
tmp_inputs = {
        "page_size": 7,
        "keyword_map": {'conditions':[fin_condition], 
                        'treatments':[treatements_eng[0]]
                       },
        "keywords": {
            "OPERATOR": 'AND'
        }
}

time.sleep(.5)
response = search_api.build_search_query_and_get_pmid(tmp_inputs, 
                                                      api_key=pubmed_api_key)
response

(['38711893',
  '40369167',
  '41213063',
  '40211189',
  '40526090',
  '40140597',
  '41268440'],
 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=(Colorectal cancer)+AND+(Crizotinib)&retmax=7&retmode=json&api_key=6892c4129cef143ff92d11533848d2e0d908',
 '67')

In [7]:
df_papers = pmid2papers(#pmid_list=['40526090', '41213063', '41268440'],
                        pmid_list=response[0], 
                        api_key=pubmed_api_key)
df_papers[0]

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract
0,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...
1,40369167,Cell death and differentiation,2025,Nov,,ARID1A loss enhances sensitivity to c-MET inhi...,Journal Article,"Xu Zhang, Zihuan Wang, Yilin He, Kejin Wang, C...","ARID1A, a subunit of the SWI/SNF chromatin-rem..."
2,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...
3,40211189,BMC cancer,2025,Apr,10,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"Journal Article, Clinical Trial, Phase I","Francesca Aroldi, Elena Elez, Thierry André, G...",Targeting RAS mutant (MT) colorectal cancer (C...
4,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...
5,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...
6,41268440,Frontiers in pharmacology,2025,,,Case Report: HGF and NF1 mutations as putative...,"Case Reports, Journal Article","Zhitao Chen, Shan Luo, Yangjun Gu, Qiyong Li",Hepatocellular carcinoma (HCC) is a highly agg...


### papers screen criterias

In [8]:
# test the apis

api = ScreeningCriteriaGeneration()
ecs_p = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=2,
    num_abstract_criteria=2,
    thinking=False,
)
ecs_p

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer diagnosis confirmed by histopathology or imaging",
        "Studies must report Crizotinib as an intervention used in treatment",
        "Comparative studies involving Crizotinib versus other treatments or placebo are eligible",
        "Outcomes related to efficacy, toxicity, survival rates, or quality of life are relevant",
        "Study designs including randomized controlled trials, cohort studies, or case-control studies are acceptable"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'Crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there evidence of Crizotinib administration in the study population?",
        "Are outcomes such as response rate, progression-free survival, overall survival, or adverse

{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title mention 'Crizotinib'?"],
 'content_criteria': ['Is there evidence of Crizotinib administration in the study population?',
  'Are outcomes such as response rate, progression-free survival, overall survival, or adverse events reported?'],
 'eligibility_analysis': ['Patients must have colorectal cancer diagnosis confirmed by histopathology or imaging',
  'Studies must report Crizotinib as an intervention used in treatment',
  'Comparative studies involving Crizotinib versus other treatments or placebo are eligible',
  'Outcomes related to efficacy, toxicity, survival rates, or quality of life are relevant',
  'Study designs including randomized controlled trials, cohort studies, or case-control studies are acceptable']}

In [9]:
papers = df_papers[0]["Title"] + ": " + df_papers[0]["Abstract"].fillna("") # important to fillna
papers = papers.tolist()
papers[:1]

['Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (\nA 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a\nThis study reported a co-existence of a']

In [10]:
'''
'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title mention 'Crizotinib'?"],
'content_criteria': ['Is there evidence of Crizotinib administration in the study methods section?',
  'Are outcomes related to patient response, adverse events, or survival discussed in the results section?'],
'''
title_criteria = ecs_p['title_criteria']
content_criteria = ecs_p['content_criteria']
title_criteria+content_criteria

["Does the title mention 'colorectal cancer'?",
 "Does the title mention 'Crizotinib'?",
 'Is there evidence of Crizotinib administration in the study population?',
 'Are outcomes such as response rate, progression-free survival, overall survival, or adverse events reported?']

### __check

In [15]:
from trialmind.llm import _batch_inputs_to_messages

In [22]:
from trialmind.prompts.screening import LITERATURE_SCREENING_FC
population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
intervention = f"{treatements_eng[0]}",
comparator = "",
outcome = "",
criteria = title_criteria + content_criteria
criteria_text = [f"{idx+1}. {c}" for idx, c in enumerate(criteria)]
n_criteria = len(criteria_text)
    
batch_inputs = []
for paper in papers[:1]:
    batch_inputs.append({
        "P": population,
        "I": intervention,
        "C": comparator,
        "O": outcome,
        "paper_content": paper,
        "criteria_text": criteria_text,
        "num_criteria": n_criteria
    })
            
LITERATURE_SCREENING_FC

'\n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.\n\n# IMPORTANT NOTE #\nIf the information within the provided paper content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.\n\n# PICO FRAMEWORK #\n- P (Patient, Problem or Population): {P}\n- I (Intervention): {I}\n- C (Comparison): {C}\n- O (Outcome): {O}\n\n# PAPER DETAILS #\n- Provided Paper: {paper_content}\

In [37]:
tools

[{'type': 'function',
  'function': {'name': 'PaperEvaluation',
   'description': '',
   'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4',
      'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'},
     'rationale': {'description': 'A rationale for each criteria evaluation',
      'items': {'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'}},
    'required': ['evaluations', 'rationale'],
    'type': 'object'}}}]

In [45]:
messages = _batch_inputs_to_messages(prompt_template=LITERATURE_SCREENING_FC, batch_inputs=batch_inputs)

from openai import AsyncOpenAI
tools = [{'type': 'function', 'function': {'name': 'PaperEvaluation', 'description': '', 'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4', 'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}, 'rationale': {'description': 'A rationale for each criteria evaluation', 'items': {'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}}, 'required': ['evaluations', 'rationale'], 'type': 'object'}}}]

async_openai_client = AsyncOpenAI(
        base_url=os.getenv("BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        http_client=httpx.AsyncClient(verify=False)
    )
response = await async_openai_client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=messages[0],
    tools=tools,
    temperature=0
)
response.choices[0].message

ChatCompletionMessage(content='```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated 

In [52]:
response.choices[0].message.content

'```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated in the title.\n\n4. **Are the 

In [60]:
from trialmind.api import extract_json,parse_json_outputs

In [62]:
parse_json_outputs([response.choices[0].message.content])

[{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["The title mentions 'metastatic colorectal cancer'",
   "The title mentions 'Crizotinib'",
   'There is no comparison mentioned in the title',
   'Methods and results sections are briefly described but not detailed enough to confirm clarity',
   'The text indicates adenocarcinoma as the type of colorectal cancer',
   'No explicit statement about outcome duration is made']}]

### 5. paper screen


In [11]:

api = LiteratureScreening()
ec_predP = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = title_criteria + content_criteria,
    papers = papers, # make for the top-100 for demo
)
ec_predP

1

parsed_results in asynch ['```json\n{\n    "evaluations": ["YES", "YES", "YES", "UNCERTAIN"],\n    "rationale": [\n        "Title mentions \'Colorectal cancer\'",\n        "Title mentions \'Crizotinib\'",\n        "Study population includes patients treated with Crizotinib",\n        "No explicit reporting of outcomes like response rate, progression-free survival, overall survival, or adverse events"\n    ]\n}\n```']

parsed_results in asynch ['```json\n{\n    "evaluations": ["YES", "YES", "YES", "UNCERTAIN"],\n    "rationale": [\n        "The title mentions \'Colorectal cancer\'",\n        "The title mentions \'Crizotinib\'",\n        "There is evidence of Crizotinib administration in the study population",\n        "No outcomes such as response rate, progression-free survival, overall survival, or adverse events are reported"\n    ]\n}']

parsed_results in asynch ['```json\n{\n    "evaluations": ["YES", "YES", "YES", "UNCERTAIN"],\n    "rationale": [\n        "Title mentions \'col

[{'evaluations': ['YES', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["Title mentions 'Colorectal cancer'",
   "Title mentions 'Crizotinib'",
   'Study population includes patients treated with Crizotinib',
   'No explicit reporting of outcomes like response rate, progression-free survival, overall survival, or adverse events']},
 {'evaluations': ['YES', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["The title mentions 'Colorectal cancer'",
   "The title mentions 'Crizotinib'",
   'There is evidence of Crizotinib administration in the study population',
   'No outcomes such as response rate, progression-free survival, overall survival, or adverse events are reported']},
 {'evaluations': ['YES', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["Title mentions 'colorectal cancer'",
   "Title mentions 'Crizotinib'",
   'Study population includes patients treated with Crizotinib',
   'No explicit reporting of response rate, progression-free survival, overall survival, or adverse events']},
 {'eval

In [12]:
evalsP = [i['evaluations'] for i in ec_predP]

word2int = {"YES": 1, 
            "UNCERTAIN": 0,
            "NO": -1
           }
#rev_subs = { v:k for k, v in word2int.items()} # subs.iteritems() In Python 3
new_evalsP = []
for one_e in evalsP:
    new_evalsP.append([word2int.get(item, 0) for item in one_e ])
new_evalsP = np.array(new_evalsP)    
print(new_evalsP)
print(new_evalsP.sum(axis=1))
df_papers[0]['screen_eval'] = new_evalsP.sum(axis=1)
df_papers[0]

[[ 1  1  1  0]
 [ 1  1  1  0]
 [ 1  1  1  0]
 [ 1  1  1 -1]
 [ 1  1  1  0]
 [ 1  1  1  0]
 [-1  1 -1  0]]
[ 3  3  3  2  3  3 -1]


,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract,screen_eval
0,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...,3
1,40369167,Cell death and differentiation,2025,Nov,,ARID1A loss enhances sensitivity to c-MET inhi...,Journal Article,"Xu Zhang, Zihuan Wang, Yilin He, Kejin Wang, C...","ARID1A, a subunit of the SWI/SNF chromatin-rem...",3
2,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...,3
3,40211189,BMC cancer,2025,Apr,10,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"Journal Article, Clinical Trial, Phase I","Francesca Aroldi, Elena Elez, Thierry André, G...",Targeting RAS mutant (MT) colorectal cancer (C...,2
4,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...,3
5,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...,3
6,41268440,Frontiers in pharmacology,2025,,,Case Report: HGF and NF1 mutations as putative...,"Case Reports, Journal Article","Zhitao Chen, Shan Luo, Yangjun Gu, Qiyong Li",Hepatocellular carcinoma (HCC) is a highly agg...,-1


### 6. papers results extraction
Только для статей с оценкой выше 0 !! Чтобы не загружать полный текст у всех

+ надо посмотреть: если фул текст недоступен, то загружается ли абстракт

In [13]:
pmid_list = df_papers[0][df_papers[0].screen_eval>=0].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers0 = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers0.append(paper_content)

df_papers[0]['FullText'] = ''
df_papers[0].loc[df_papers[0].screen_eval>=0, 'FullText'] = papers0
df_papers[0]

https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/38711893/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40369167/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41213063/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40211189/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40526090/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40140597/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908


,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract,screen_eval,FullText
0,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...,3,Partial response to crizotinib + regorafenib +...
1,40369167,Cell death and differentiation,2025,Nov,,ARID1A loss enhances sensitivity to c-MET inhi...,Journal Article,"Xu Zhang, Zihuan Wang, Yilin He, Kejin Wang, C...","ARID1A, a subunit of the SWI/SNF chromatin-rem...",3,ARID1A loss enhances sensitivity to c-MET inhi...
2,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...,3,Radiotherapy-Related Autophagy Genes Predict P...
3,40211189,BMC cancer,2025,Apr,10,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"Journal Article, Clinical Trial, Phase I","Francesca Aroldi, Elena Elez, Thierry André, G...",Targeting RAS mutant (MT) colorectal cancer (C...,2,A Phase Ia/b study of MEK1/2 inhibitor binimet...
4,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...,3,Case Report WIN-MTB-2023001 WIN International ...
5,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...,3,A phase Ia study of the MEK1/2 inhibitor PD-03...
6,41268440,Frontiers in pharmacology,2025,,,Case Report: HGF and NF1 mutations as putative...,"Case Reports, Journal Article","Zhitao Chen, Shan Luo, Yangjun Gu, Qiyong Li",Hepatocellular carcinoma (HCC) is a highly agg...,-1,


In [14]:
splited = split_text_into_chunks(papers[0], chunk_size=600,
                                             chunk_overlap=300)
splited

['Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (\nA 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a\nThis study reported a co-existence of a']

In [15]:
papers0[2]

'Radiotherapy-Related Autophagy Genes Predict Prognosis and Reveal Immunoscape Features and Immunotherapeutic Agents in Colorectal Cancer Patients.\nNumerous genes have been associated with colorectal cancer (CRC) treatment in prior studies, but the impact of radiotherapy-related autophagy genes (RRAGs) on CRC remains largely unexplored. The Cancer Genome Atlas (TCGA) and Gene Expression Omnibus (GEO) databases were implemented to screen out RRAGs significantly associated with survival, and a prognostic model was constructed. Samples were categorized into high- and low-risk groups with median riskscore. Immunomicroenvironment analysis, immunotherapy response prediction, enrichment analysis, tumor mutation analysis and drug prediction were performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative reverse transcription polymerase chain reaction (qRT-PCR). There were 10 CRC-related RRAGs found. Prognostic models were built with RRAGs. Based on im

In [ ]:
{
     "fieldresult": [
       {
         "name": "Title",
         "value": "Основы программирования на Python",
         "source_id": [104]
       },
       {
         "name": "Pages",
         "value": "300 страниц",
         "source_id": [105]
       }
     ]
   }

In [ ]:

        Format:
        ```json
        [
            {{
                "name":  \\ str, length <= 25 tokens
                "value":  \\ str, length <= 25 tokens
                "source_id":  \\ list[int], length <= 3 ids
            }},
            {{
                "name":, \\ str, length <= 25 tokens
                "value":  \\ str, length <= 25 tokens
                "source_id":  \\ list[int], length <= 3 ids
            }},
            ...
        ]
        ```
    \"\"\"
```
    
    
new
```json
        [
            {{
                "name": "fieldName1",
                "value": "extractedInfo1",
                "source_id":[source1, source2, ...]
            }},
            {{
                "name":["fieldName2"],
                "value":["extractedInfo2"],
                "source_id":[source1, source2, ...]
            }},
            ...
        ]
```

In [26]:
from pydantic import BaseModel, validator, Field, conlist  # This is the new version
from typing import Dict, Literal
class FieldResult(BaseModel):
    name: str = Field(description='Field name that accurately represents the content of the field based on its description.',
                     max_length=25)
    value: str = Field(description='Extracted information from the text based on the field description.',
                      max_length=50)
    source_id: conlist(int,min_length=1, max_length=3) = Field(description='Cited document IDs.')
    
class Results(BaseModel):
    fieldresult: FieldResult
    
Results.model_json_schema()

{'$defs': {'FieldResult': {'properties': {'name': {'description': 'Field name that accurately represents the content of the field based on its description.',
     'maxLength': 25,
     'title': 'Name',
     'type': 'string'},
    'value': {'description': 'Extracted information from the text based on the field description.',
     'maxLength': 50,
     'title': 'Value',
     'type': 'string'},
    'source_id': {'description': 'Cited document IDs.',
     'items': {'type': 'integer'},
     'maxItems': 3,
     'minItems': 1,
     'title': 'Source Id',
     'type': 'array'}},
   'required': ['name', 'value', 'source_id'],
   'title': 'FieldResult',
   'type': 'object'}},
 'properties': {'fieldresult': {'$ref': '#/$defs/FieldResult'}},
 'required': ['fieldresult'],
 'title': 'Results',
 'type': 'object'}

In [69]:
from pydantic import BaseModel, validator, Field, conlist,   # This is the new version
from typing import Dict, Literal
import pydantic
import random
class Nested(pydantic.BaseModel):
    a: str = pydantic.Field(examples=["hello"])


class ExampleData(pydantic.BaseModel):
    a: int = pydantic.Field(examples=[1])
    b: str = pydantic.Field(min_length=10, max_length=10, examples=["abc-123-45"])
    c: int = pydantic.Field(default=100)
    # Note: Example will be populated by child, so no explicit example needs
    # to be provided for nested models
    n: Nested
    
nn = 1    
class FieldResult(BaseModel):
    name: str = Field(description='Field name that accurately represents the content of the field based on its description.',
                     max_length=25, default='effectiveness')
    value: str = Field(description='Extracted information from the text based on the field description.',
                      max_length=50, default='total of 5% died')
    #source_id: conlist(int,min_length=1, max_length=3) = Field(description='Cited document IDs.', examples=[1,3])
    
class Results(BaseModel):
    fieldresult: list[FieldResult]

    
def populate_with_examples_recursive(model: pydantic.BaseModel) -> dict:
    """Populate fields with example values"""

    result = {}
    
    for name, field_info in model.model_fields.items():
        if issubclass(field_info.annotation, pydantic.BaseModel):
            result[name] = populate_with_examples_recursive(field_info.annotation)
        elif field_info.examples:
            result[name] = random.choice(field_info.examples)
        elif field_info.default:
            result[name] = field_info.default

    return dict(result)


# Execute it:
#json.loads(f'''{populate_with_examples_recursive(ExampleData)}''')

(populate_with_examples_recursive(Results))

SyntaxError: trailing comma not allowed without surrounding parentheses (1178800490.py, line 1)

In [83]:
class FieldResult(BaseModel):
    name: str = Field(description='Field name that accurately represents the content of the field based on its description.',
                     max_length=25, default='effectiveness')
    value: str = Field(description='Extracted information from the text based on the field description.',
                      max_length=50, default='total of 5% died')
    source_id: list[int] = Field(description='Cited document IDs.', examples=[1,3])
    #conlist ,min_length=1, max_length=3
class Results(BaseModel):
    fieldresult: list[FieldResult]

a1 = FieldResult()
a = Results(fieldresult=[a1,a1])

print(a.model_dump_json(indent=4))

ValidationError: 1 validation error for FieldResult
source_id
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [27]:
papers[:2]

['Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (\nA 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a\nThis study reported a co-existence of a',
 'ARID1A loss enhances sensitivity to c-MET inhibition by dual targeting of GPX4 and iron homeostasis, inducing ferroptosis.: ARID1A, a subunit of the SWI/SNF chromatin-remodeling complex, functions as a tumor suppressor in various cancer types. Owing to its high frequency of inactivating mutations, ARID1A has emerged as a promising target for the development of anticancer drugs. In this study, we report that ARID1A-deficient colorectal cancer (CRC) cells induce synthetic lethality when treated with inhibitors of c-MET receptor tyrosine kinase. c-MET 

['',
 'Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (\nA 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a\nThis study reported a co-existence of a',
 'ARID1A loss enhances sensitivity to c-MET inhibition by dual targeting of GPX4 and iron homeostasis, inducing ferroptosis.: ARID1A, a subunit of the SWI/SNF chromatin-remodeling complex, functions as a tumor suppressor in various cancer types. Owing to its high frequency of inactivating mutations, ARID1A has emerged as a promising target for the development of anticancer drugs. In this study, we report that ARID1A-deficient colorectal cancer (CRC) cells induce synthetic lethality when treated with inhibitors of c-MET receptor tyrosine kinase. c

In [54]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=['',*papers[:2]],
    fields=['Crizotinib Results, string, main results of the study regarding Crizotinib about treatment effectiveness',
           'Participants, string, number of participants'],#api.DEFAULT_FIELDS,
    llm=os.getenv("MODEL_NAME"),
    chunk_size=600,
    chunk_overlap=300,
    thinking=False,
)
extracted

You are now the following python function: ```
def extract_fields_from_input_study(inputs: Dict[str, Any]) -> str:
    """
    This function is tasked with analyzing clinical trial study reports or papers to extract specific information as structured data and provide citations for the extracted information.
    The user will provide a list of fields they are interested in, along with a natural language description for each field to guide you on what content to look for and from which parts of the report to extract it.

    IMPORTANT:
    For each field described by the user, you need to:
    1. Identify and extract the relevant information from the report, based on the provided description.
    2. Generate a field name that accurately represents the content of the field based on its description.
    3. Structure the extracted information into a standard format whenever possible (e.g., integer, numerical values, dates, keywords, list of terms). 
        If standardization is not possibl

[[{'name': 'Crizotinib_Results',
   'value': 'NP',
   'source_id': [],
   'cited_blocks': []},
  {'name': 'Participants',
   'value': 'NP',
   'source_id': [],
   'cited_blocks': []}],
 [{'name': 'Crizotinib Results',
   'value': 'main results of the study regarding Crizotinib about treatment effectiveness',
   'source_id': ['0'],
   'cited_blocks': ['Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (\nA 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a\nThis study reported a co-existence of a']},
  {'name': 'Participants',
   'value': 'number of participants',
   'source_id': ['0'],
   'cited_blocks': ['Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal canc

In [56]:
os.environ["BASE_URL"] = "https://router.huggingface.co/v1"
os.environ["MODEL_NAME"] ='Qwen/Qwen3-32B'
os.environ["OPENAI_API_KEY"] = os.getenv("HUGGINGFACE_HUB_TOKEN")

In [59]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=['',*papers[:2]],
    fields=['Crizotinib Results, string, main results of the study regarding Crizotinib about treatment effectiveness',
           'Participants, string, number of participants'],#api.DEFAULT_FIELDS,
    llm=os.getenv("MODEL_NAME"),
    chunk_size=600,
    chunk_overlap=300,
    thinking=True,
)
extracted

You are now the following python function: ```
def extract_fields_from_input_study(inputs: Dict[str, Any]) -> str:
    """
    This function is tasked with analyzing clinical trial study reports or papers to extract specific information as structured data and provide citations for the extracted information.
    The user will provide a list of fields they are interested in, along with a natural language description for each field to guide you on what content to look for and from which parts of the report to extract it.

    IMPORTANT:
    For each field described by the user, you need to:
    1. Identify and extract the relevant information from the report, based on the provided description.
    2. Generate a field name that accurately represents the content of the field based on its description.
    3. Structure the extracted information into a standard format whenever possible (e.g., integer, numerical values, dates, keywords, list of terms). 
        If standardization is not possibl

AttributeError: 'str' object has no attribute 'get'

# d

In [81]:
pmid_list = df_papers[0].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers0 = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers0.append(paper_content)


https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40140597/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40211189/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40526090/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41213063/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41268440/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/38711893/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40369167/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908


In [ ]:
df_papers[0]

In [85]:
papers0[2]

'Case Report WIN-MTB-2023001 WIN International Molecular Tumor Board A 62-year-old male with metastatic colorectal cancer with 5 prior lines of treatment\nHeavily pretreated metastatic colorectal cancer (mCRC) poses significant therapeutic challenges. Advances in molecular profiling enables personalized strategies. We present a 62-year-old male with mCRC harboring BRAF, MET, APC, TP53 and NRAS alterations, following FOLFOX and FOLFIRI, dabrafenib plus panitumumab, and a BRAF inhibitor clinical trial, each leading to initial responses followed by disease progression.\nWIN Consortium International Molecular Tumor Board (MTB), included experts from institutions across 13 countries. Enrollment in suitable clinical trials was explored but limited by availability. Personalized combinations suggested included amivantamab-vmjw (anti-MET/EGFR antibody) (one-third standard dose) (for MET amplification and due to prior response to anti-EGFR antibody), trametinib, 1 mg po daily (MEK inhibitor for 

In [11]:
[len(i) for i in papers]

[1672, 1652, 27171, 2627, 46819]

In [15]:
splited = split_text_into_chunks(papers[1], chunk_size=600,
                                             chunk_overlap=300)
splited

['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.',
 'Background: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic',
 'challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression

In [12]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers,
    fields=['Results, string, main results of the study regarding Crizotinib, usually about treatment effectiveness'],#api.DEFAULT_FIELDS,
    llm='Qwen/Qwen3-32B',#"openai-gpt-4o",
    chunk_size=600,
    chunk_overlap=300,
    thinking=False,
)
extracted

False


[[{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [2],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify mutations associated with resistance. Results: NGS revealed the loss of MET amplification and identified HGF and NF1 mutations as potential bypass mechanisms. Specifically,']}],
 [{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [5],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative rev

[[{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [4],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative reverse transcription polymerase chain reaction (qRT-PCR). There were 10 CRC-related RRAGs found. Prognostic models were built with RRAGs. Based on immune infiltration analysis, low-risk populations showed significantly greater levels of immune infiltration (P P ARHGEF17 mutation rate was 6%. Medications such as CGP-082996, Dasatinib, Erlotinib, and Salubrinal were more sensitive to high-risk group, whereas drugs such as FTI-277, DMOG, and Crizotinib were more sensitive to low-risk group.']}],
 [{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [1],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in ME

In [38]:
llm_reasoning = ChatOpenAI(
    base_url=BASE_URL,
    api_key=HUGGINGFACE_HUB_TOKEN,
    model=MODEL_NAME_HF,
    streaming=True,
    temperature=0,
    #http_client=httpx_client
    http_async_client=httpx.AsyncClient(verify=False)
)

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '/no_think {text}'),
    # ("ai", "Good.")
])

chain = prompt | llm_reasoning
response = await chain.ainvoke({"text": ff})

#fin_condition = response.content.strip('<think>\n\n</think>\n\n')
print(response.content)

<think>

</think>

```json
[
    {
        "name": "Crizotinib effectiveness",
        "value": "More sensitive to low-risk group",
        "source_id": [3]
    }
]
```
